# Reading in Kununu and Glassdoor reviews, Formatting and Appending 
------------------
> <i>Description: In this notebook, We are formatting and Concatinating Kununu and Glassdoor translated data so that we can use it for text classification and sentiment determination.</i>

Input Files: 
1) Kununu_reviews_translated.xlsx
2) Glassdoor_reviews_translated.xlsx

Output:
1) reviews_merged.csv

We map the position and department into common buckets for Kununu and Glassdoor:

Positions:

* Employee

* Apprentice
* Contractor
* Manager
* Student
* Intern
* Freelancer

Departments:

* Corporate

* Retail
* Logistics





In [87]:
import pandas as pd
import ast
import numpy as np

### Reading translated Kununu and glassdoor, Country mapping, formatting Date and creating a feature called year for later

In [88]:
kununu = pd.read_excel('Kununu_reviews_translated_barbara.xlsx')
glassdoor = pd.read_excel('Glassdoor_reviews_translated_barbara.xlsx')
country = pd.read_csv('useTheCountry.csv')

kununu = kununu.rename(columns={'createdAt': 'date'})
kununu['date'] = pd.to_datetime(kununu['date'])
kununu['year'] = kununu['date'].dt.year

glassdoor['date'] = pd.to_datetime(glassdoor['reviewDateTime'], format= 'mixed')
glassdoor['year'] = glassdoor['date'].dt.year

### Deconstructing pros, cons and suggestions form Kununu file

For Kununu: Positive, Negative and Suggestion were in dictionary inside feature text, we extract these components and make 3 different columns for each

In [89]:
def extract_text_components(text_data):
    positive_text = None
    negative_text = None
    suggestion_text = None
    # converting string to list
    text_data = ast.literal_eval(text_data)

    if isinstance(text_data, list):
        for component in text_data:
            if component['id'] == 'positive':
                positive_text = component['text']
            elif component['id'] == 'negative':
                negative_text = component['text']
            elif component['id'] == 'suggestion':
                suggestion_text = component['text']

    return positive_text, negative_text, suggestion_text

def extract_components_from_dataframe(df, column_name):
    df['pros'] = df[column_name].apply(lambda x: extract_text_components(x)[0])
    df['cons'] = df[column_name].apply(lambda x: extract_text_components(x)[1])
    df['suggestion'] = df[column_name].apply(lambda x: extract_text_components(x)[2])
    return df

kununu = extract_components_from_dataframe(kununu, "texts_translated")

### Mapping department and position ranking for Kununu

Purpose for doing this is to streamline the data so that when we merge both Kununu and Glassdoor the features as well as data in them are in similar format.

In [ ]:
position_mapping = {
    'employee': 1,
    'apprentice': 2,
    'contractor': 3,
    'manager': 4,
    'student': 5,
    'intern': 6,
    'freelancer': 7
}

kununu['position_code'] = kununu['position'].map(position_mapping)

def map_department_to_function(department):
    corporate_departments = [
        'procurement', 'controlling', 'legal', 'recruiting', 'it', 'administration', 
        'design', 'product', 'communication', 'research', 'management'
    ]
    retail_departments = ['sales']
    logistics_departments = ['logistic', 'operations']
    
    if department in corporate_departments:
        return 'Corporate'
    elif department in retail_departments:
        return 'Retail'
    elif department in logistics_departments:
        return 'Logistics'
    else:
        return np.nan  

kununu['department'] = kununu['department'].apply(map_department_to_function)

### Mapping Departments and position ranking for Glassdoor

Mapping all the different position into 7 different buckets so that it matches the format of Kununu. 

In [90]:
position_mapping = {
    'employee': ['Sales Associate', 'Vendedor', 'Salesman', 'Retail Sales Assistant', 'Sales Consultant', 'Sales Representative', 'Sales Assistant'],
    'apprentice': ['Stagista', 'Practicante', 'Jovem Aprendiz', 'Intern', 'Internship', 'Praktikant', 'Praktikantin', 'Estagiário'],
    'contractor': ['Consultant', 'Technical Consultant', 'Expert IT Consultant'],
    'manager': ['Manager', 'HR Manager', 'Project Manager', 'Store Manager', 'Sales Manager', 'Finance Manager', 'Team Leader'],
    'student': ['Graduate Student', 'Working Student', 'Studentische Aushilfe'],
    'intern': ['Intern', 'Internship', 'Marketing Intern', 'Finance Intern', 'Wholesale Intern', 'Product Design Intern'],
    'freelancer': ['Freelancer', 'Contractor']
}

def map_position(title):
    if pd.isna(title): 
        return np.nan
    title = str(title)  
    for position, keywords in position_mapping.items():
        if any(keyword.lower() in title.lower() for keyword in keywords):
            return position
    return np.nan  

glassdoor['position'] = glassdoor['jobTitle.text'].apply(map_position)

position_mapping = {
    'employee': 1,
    'apprentice': 2,
    'contractor': 3,
    'manager': 4,
    'student': 5,
    'intern': 6,
    'freelancer': 7
}

glassdoor['position_code'] = glassdoor['position'].map(position_mapping)

### Mapping Departments for glassdoor

In [91]:
corporate_keywords = [
    # English Keywords
    "Manager", "Director", "Vice President", "President", "CEO", "CFO", "COO", "Chief", 
    "Executive", "Global", "Corporate", "Business Analyst", "Financial Analyst", 
    "Business Partner", "Business Development", "Strategy", "Strategic", "Marketing", 
    "Communications", "Human Resources", "HR", "Legal", "Compliance", "Procurement", 
    "Sustainability", "Product Manager", "Product Owner", "Project Manager", "Consultant", 
    "Business Operations", "Content Production", "Graphic Design", "UX/UI", "SEO", 
    "Performance Marketing", "Creative", "Digital", "Brand", "Social Media", "Event Manager", 
    "Franchise Manager", "Talent Acquisition", "Learning & Development", "eCommerce", 
    "Customer Experience", "CRM", "Training", "Planning", "Buyer", "Data Analyst", 
    "Risk Management", "Corporate Affairs", "Merchandising", "Finance", "Tax", "Audit", 
    "Real Estate", "Architecture", "Supply Chain", "Corporate", "Marketing",
    
    # German Keywords (Corporate)
    "Geschäftsführer", "Bereichsleiter", "Abteilungsleiter", "Manager", "Direktor", 
    "Berater", "Analyst", "Unternehmensstrategie", "Marketing", "Personalwesen", 
    "Rechtsabteilung", "Nachhaltigkeit", "Produktmanager", "Projektleiter", "Kundenerfahrung", 
    "Markenstrategie", "Personalentwicklung", "Finanz", "Rechnungswesen", "Compliance", 
    "Einkauf", "Content-Produktion", "Digital",
    
    # French Keywords (Corporate)
    "Directeur", "Manager", "Responsable", "Consultant", "Chef", "Stratégie", "Analyste", 
    "Marketing", "Ressources Humaines", "Finance", "Audit", "Comptabilité", 
    "Développement des affaires", "Communications", "Juridique", "Conformité", "Achats", 
    "Gestion des risques", "Apprentissage", "Acquisition de talents", "Formation", 
    "Chef de produit", "Gestionnaire de projet", "Production de contenu", "eCommerce", 
    "Expérience client",
    
    # Spanish Keywords (Corporate)
    "Gerente", "Director", "Responsable", "Consultor", "Estrategia", "Analista", 
    "Recursos Humanos", "Cumplimiento", "Marketing", "Finanzas", "Compras", 
    "Desarrollo de Negocios", "Comunicaciones", "Legal", "Producción de Contenidos", 
    "Cadenas de Suministro", "Arquitectura", "Planificación", "Comercio electrónico", 
    "Experiencia del Cliente", "Estrategia de marca"
]

retail_keywords = [
    # English Keywords
    "Store Manager", "Retail Manager", "Sales Associate", "Cashier", "Customer Service", 
    "Retail Operations", "Assistant Store Manager", "Department Manager", "Area Manager", 
    "Retail", "Retail Sales", "Salesperson", "Sales Advisor", "Tailor", "Shop Supervisor", 
    "Supervisor", "Floor Manager", "Team Leader", "Retail Sales Assistant", "Sale", 
    "Retail", "Sales", "Selling",
    
    # German Keywords (Retail)
    "Filialleiter", "Verkäufer", "Einzelhandel", "Kassierer", "Ladenleiter", 
    "Kundendienst", "Abteilungsleiter", "Verkaufsberater", "Teamleiter", "Ladenaufsicht", 
    "Einzelhandelsmanagement",
    
    # French Keywords (Retail)
    "Responsable de Magasin", "Vendeur", "Caisse", "Assistant Commercial", 
    "Conseiller de Vente", "Superviseur", "Gestion de Magasin", "Service Clients", 
    "Magasinier", "Manager de Magasin", "Responsable de Rayon", "Chef de Magasin", 
    "Commis de Vente",
    
    # Spanish Keywords (Retail)
    "Gerente de Tienda", "Cajero", "Asistente de Ventas", "Vendedor", 
    "Asociado de Ventas", "Supervisor de Tienda", "Jefe de Departamento", 
    "Servicio al Cliente", "Gestión de Ventas al por Menor", "Encargado de tienda"
]

logistics_keywords = [
    # English Keywords
    "Warehouse Manager", "Warehouse Worker", "Logistics", "Logistics Manager", 
    "Supply Chain", "Operations", "Transportation", "Customs", "Logistics Coordinator", 
    "Inventory", "Planner", "Warehouse Supervisor", "Distribution", "Shipping", 
    "Technician", "Transport", "Freight", "Fleet", "Driver", "Logistics Planner", 
    "Operations Manager", "Logistics Specialist", "Logistics Operations", 
    "Warehouse Associate", "Dispatch",
    
    # German Keywords (Logistics)
    "Lagerleiter", "Lagerarbeiter", "Logistik", "Logistikleiter", "Supply Chain", 
    "Transport", "Zoll", "Logistikkoordinator", "Fracht", "Versand", "Techniker", 
    "Disponent", "Flottenmanager", "Transportplaner", "Lager",
    
    # French Keywords (Logistics)
    "Responsable Logistique", "Magasinier", "Transport", "Douanes", 
    "Planificateur Logistique", "Chauffeur", "Technicien Logistique", 
    "Gestionnaire de Stock", "Expédition", "Flotte", "Distribution", 
    "Coordination Logistique",
    
    # Spanish Keywords (Logistics)
    "Gerente de Logística", "Almacén", "Técnico en Logística", "Transporte", 
    "Aduanas", "Coordinador de Logística", "Planificador de Logística", 
    "Conductor", "Supervisor de Almacén", "Despacho", "Cadena de Suministro", 
    "Envíos", "Inventario", "Distribución"
]

# Function to map job categories
def map_job_category(job_title):
    if pd.isna(job_title):
        return np.nan  
    job_title_lower = job_title.lower() 
    
    if any(keyword.lower() in job_title_lower for keyword in corporate_keywords):
        return "Corporate"
    
    elif any(keyword.lower() in job_title_lower for keyword in retail_keywords):
        return "Retail"
    
    elif any(keyword.lower() in job_title_lower for keyword in logistics_keywords):
        return "Logistics"
    
    return "Other"

# Apply the function to the 'jobTitle.text' column to create 'job_category' column
glassdoor['department'] = glassdoor['jobTitle.text'].apply(map_job_category)

### Extracting relevant columns, Renaming them and adding the country details 

When we concatinate Kununu and Glassdoor, the number of columns and the name of the columns must match

In [93]:
glassdoor = glassdoor[['reviewId', 'date', 'year', 'ratingOverall', 'position', 'position_code', 'department', 'pros_translated', 'cons_translated', 'advice_translated', 'summary_translated']]
glassdoor = glassdoor.rename(columns={'reviewId': 'uuid','ratingOverall': 'rating', 'pros_translated': 'pros', 'cons_translated': 'cons', 'advice_translated': 'suggestion'})
glassdoor['uuid'] = glassdoor['uuid'].astype(str)
glassdoor=pd.merge(glassdoor, country[['uuid','country']], on='uuid', how='left')
glassdoor['file'] = 'glassdoor'
print(glassdoor.shape)

kununu = kununu[['uuid', 'date','year', 'score', 'position','position_code', 'department', 'pros', 'cons', 'suggestion', 'ratings_translated', 'trainee']]
kununu = kununu.rename(columns={'score': 'rating'})
kununu=pd.merge(kununu, country[['uuid','country']], on='uuid', how='left')
kununu['file'] = 'kununu'
print(kununu.shape)

(1872, 13)
(607, 14)


ratings_translated feature in Kununu is a dictionary that contains category-wise rating and reviews like atmoshphere, team, etc.
Extracting this info and storing in concatenated_ratings

In [94]:
# Function to concatenate 'id' and 'text', removing the 'apprenticeship' prefix when applicable
def concatenate_text_from_dicts(row, is_trainee):
    # Parse the list of dictionaries from the row
    dictionaries = eval(row)
    
    concatenated_text = []
    
    for d in dictionaries:
        if 'id' in d and 'text' in d and d.get('text') is not None:
            # If trainee is present, remove 'apprenticeship' prefix from 'id'
            id_value = d['id'].replace('apprenticeship', '') if is_trainee else d['id']
            concatenated_text.append(f"{id_value}: {d['text']}")
    
    return ', '.join(concatenated_text)

# Apply the function to process 'ratings_translated' and check the 'trainee' column for each row
kununu['concatenated_ratings'] = kununu.apply(
    lambda row: concatenate_text_from_dicts(row['ratings_translated'], pd.notna(row['trainee'])), axis=1
)


### Merging and outputing to CSV

In [95]:
kununu.drop(columns='trainee', inplace= True)
reviews_merged = pd.concat([kununu, glassdoor], axis=0)
reviews_merged.shape

(2479, 15)

In [97]:
reviews_merged = reviews_merged.drop_duplicates(subset='uuid', keep='first')
print(reviews_merged.shape)
reviews_merged.to_csv('reviews_merged.csv', index=False)

(2449, 15)
